# FAIR Attributes

According to [F1 of the *FAIR Principles*](https://www.go-fair.org/fair-principles/f1-meta-data-assigned-globally-unique-persistent-identifiers/) attributes shall be assigned to globally unique and persistent identifiers.

Here's what www.go-fair.org says about it:

*"Globally unique and persistent identifiers remove ambiguity in the meaning of your published data by assigning a unique identifier to every element of metadata and every concept/measurement in your dataset. In this context, identifiers consist of an internet link (e.g., a URL that resolves to a web page that defines the concept such as a particular human protein). Many data repositories will automatically generate globally unique and persistent identifiers to deposited datasets. Identifiers can help other people understand exactly what you mean, and they allow computers to interpret your data in a meaningful way (i.e., computers that are searching for your data or trying to automatically integrate them). Identifiers are essential to the human-machine interoperation that is key to the vision of Open Science. In addition, identifiers will help others to properly cite your work when reusing your data."*

The *h5rdmtoolbox* allows assigning attributes (and their data) to identifiers. For this, each name and value of an attribute may obtain an IRI (internationalized resource identifier). The following outlines, how it is done.

## Concept

We can interpret HDF5 objects, their attribute names and attribute values as [RDF triples](https://en.wikipedia.org/wiki/Semantic_triple) (subject-predicate-object), where...
- ... a group or dataset is a *subject*
- ... the attribute name is a <u>predicate</u>
- ... and the attriute value is an **object**

In the following, we would like to describe the content of an HDF5 file. There will be a dataset or random data generated by a person, which can be identified/described by a researcher ID (ORCID).

We as humans may understand the content of such an HDF5 file. For machines to interpret the data, we need to associate [URIs](https://en.wikipedia.org/wiki/Uniform_Resource_Identifier) with the HDF5 objects. In fact, sometimes it may also not very clear to humans, what is meant with a certain attribute. To be unambiguous about it, a URI helps. Think of the attribute "contact", we will define. Is it a person or an organization? Note, that URI and [IRI](https://en.wikipedia.org/wiki/Internationalized_Resource_Identifier) may be used synonymously - IRI is built on URI by expanding the set of permitted characters.

Let's build the example step by step. We start with **creating the group "contact"**:

In [1]:
import h5rdmtoolbox as h5tbx

Failed to import module h5tbx


## Describing an HDF5 file with persistent metadata

### Example part 1: A contact person
The file is written by an author. We create a group. It contains all relevant contact data, i.e. the ORCID. The content if the group thus describes the contact person and therefore *is* a person. The group itself, however, gets the predicate *has author*:

In [2]:
with h5tbx.File(mode='w') as h5:
    grp = h5.create_group('contact', attrs=dict(orcid='https://orcid.org/0000-0001-8729-0482'))   
    grp.rdf.predicate = 'https://schema.org/author'
    grp.rdf.subject = 'http://xmlns.com/foaf/0.1/Person'  # what the content of group is, namely a foaf:Person
    grp.rdf.predicate['orcid'] =  'http://w3id.org/nfdi4ing/metadata4ing#orcid'
    grp.attrs['first_name', 'http://xmlns.com/foaf/0.1/firstName'] = 'Matthias'
    # print(grp.rdf.subject)
    h5.dump(collapsed=False)

hdf_filename = h5.hdf_filename

Using the `rdf` accessory, we can assign the objects (dataset, groups, attributes) with the internationalized resource identifier (IRI). An IRI a web resource and points to the definition in an ontology, e.g. "contact" is a "Person" and is defined in the ontology FOAF: 'http://xmlns.com/foaf/0.1/Person'. The person "has a researcher ID". This predicate is described in the M4i ([metadata4ing](https://nfdi4ing.pages.rwth-aachen.de/metadata4ing/metadata4ing/)) ontology: 'http://w3id.org/nfdi4ing/metadata4ing#orcid'

From now on, let's same some work and use the package `namespacelib`, which simplifies the work with the namespaces, so that we don't have to type the full IRI address. Some popular ones are implemented in the `rdflib` package, too:

In [3]:
from ontolutils.namespacelib import M4I, OBO, QUDT_UNIT, QUDT_KIND
from rdflib.namespace import FOAF

As a result, we can type the following:

In [4]:
M4I.orcidId  # equal to http://w3id.org/nfdi4ing/metadata4ing#orcidId

rdflib.term.URIRef('http://w3id.org/nfdi4ing/metadata4ing#orcidId')

### Example part 2: A random data dataset
Next, we add the random data dataset with units. We can even describe what type the data is. In our case it shall be velocity data. Without this specification it would otherwise not be clear to the user (or a machine):

In [5]:
import numpy as np

with h5tbx.File(hdf_filename, mode='r+') as h5:    
    ds = h5.create_dataset('grp/random_velocity', data=np.random.random(100))
    ds.attrs.create('units',
                    rdf_predicate=M4I.hasUnit,
                    data='m/s',
                    rdf_object=QUDT_UNIT.M_PER_SEC)
    ds.attrs.create('quantity_kind',
                     data='velocity',
                     rdf_predicate=M4I.hasKindOfQuantity,
                     rdf_object=QUDT_KIND.Velocity)

    h5.dump(collapsed=False)

Now, let's go further and describe how the random dataset was created and that the contact was involved in it:

In [6]:
from datetime import datetime

with h5tbx.File(hdf_filename, mode='r+') as h5:  
    proc = h5.create_group('processing_info')
    proc.rdf.subject = M4I.ProcessingStep
    proc.attrs['has_participants', OBO.has_participant] = h5['contact']
    start_time = datetime.today()
    end_time = datetime.today()
    proc.attrs.create('start_time', data=start_time,
                      rdf_predicate='https://schema.org/startTime')
    proc.attrs.create('end_time', data=end_time,
                      rdf_predicate='https://schema.org/startTime')
    proc.attrs['output', 'http://purl.obolibrary.org/obo/RO_0002234'] = h5['grp/random_velocity'].name

In [7]:
h5tbx.dump(hdf_filename, collapsed=False)

## How to make use of the FAIR HDF5 file?

There are three ways, how the above IRI assignments help us and how we might want to use the information:
1. Visual inspection by dumping the content to screen: This will outline the file (meta) content and we can click on the attributes with IRIs, which will explain the attribute (data)
2. We can extract a *JSON-LD* file. This is useful for other processes. We can also investigate this file further with tools like [JSON-LD-playground](https://json-ld.org/playground/).
3. Access IRI in (Python) code

### 1. Visual inspection

The *dump()* method will now add IRI-icons. Click on it and get redirected to the resources:

In [8]:
h5tbx.dump(hdf_filename, collapsed=False)

### 2. JSON-LD extraction

Write the JSON-LD file and share it with others or a repository. The toolbox provides `dump`-methods through the `jsonld` module. It might look a bit overwelming, however dedicated scripts can perfectly work with it while humans still can read it (with a bit of practice and patience...):

In [9]:
from h5rdmtoolbox import jsonld

In [10]:
print(
    jsonld.dumps(
        hdf_filename,
        indent=2,
        context={'m4i': 'http://w3id.org/nfdi4ing/metadata4ing#',
                 'foaf': 'http://xmlns.com/foaf/0.1/'}
    )
)

{
  "@context": {
    "foaf": "http://xmlns.com/foaf/0.1/",
    "hdf5": "http://purl.allotrope.org/ontologies/hdf5/1.8#",
    "m4i": "http://w3id.org/nfdi4ing/metadata4ing#",
    "obo": "http://purl.obolibrary.org/obo/",
    "schema": "https://schema.org/"
  },
  "@graph": [
    {
      "@id": "_:N4703badf193b4014b111bd1cec75d7b4",
      "@type": "hdf5:File",
      "hdf5:rootGroup": {
        "@id": "_:Nd003455b1263430dab801e5aa5fd7041",
        "@type": "hdf5:Group",
        "hdf5:member": [
          {
            "@id": "_:Nf33f6c231eb044bb857f26ada06e1674",
            "@type": [
              "hdf5:Group",
              "foaf:Person"
            ],
            "foaf:firstName": "Matthias",
            "hdf5:attribute": [
              {
                "@id": "_:N13f1f202e6e4403dafeb41505f8021ea",
                "@type": "hdf5:Attribute",
                "hdf5:name": "first_name",
                "hdf5:value": "Matthias"
              },
              {
                "@id": "_:

C:\Users\Matthias\Documents\programming\GitHub\h5RDMtoolbox\h5rdmtoolbox\wrapper\jsonld.py:668: UserWarning: Could not serialize <HDF5 group "/contact" (0 members)> to JSON. Will apply str(). Error: Object of type Group is not JSON serializable
  warnings.warn(f'Could not serialize {av} to JSON. Will apply str(). Error: {e}')
C:\Users\Matthias\Documents\programming\GitHub\h5RDMtoolbox\h5rdmtoolbox\wrapper\jsonld.py:668: UserWarning: Could not serialize <HDF5 dataset "random_velocity": shape (100,), type "<f8"> to JSON. Will apply str(). Error: Object of type Dataset is not JSON serializable
  warnings.warn(f'Could not serialize {av} to JSON. Will apply str(). Error: {e}')


Dump it to the file rather than to the screen:

In [11]:
with open('hdf_meta.jsonld', 'w') as f:
    jsonld.dump(hdf_filename, f, indent=2,
        context={'m4i': 'http://w3id.org/nfdi4ing/metadata4ing#',
                 'foaf': 'http://xmlns.com/foaf/0.1/'})
                # context={'foaf': 'http://xmlns.com/foaf/0.1/',
                #                                    'm4i': 'http://w3id.org/nfdi4ing/metadata4ing#'})

## 3. Access IRI in code

You may want to access the IRI of an attribute with Python within the HDF5 file. E.g. while working with the file, you may ask "Hey, what is 'contact' exactly?" or "What does the attribute 'orcid' mean?"

In [12]:
with h5tbx.File(hdf_filename) as h5:
    person_iri = h5.contact.rdf.subject
    orcid_iri = h5.contact.rdf.predicate['orcid']

... Well "contact" is a "Person" defined by the FOAF ontology:

In [13]:
person_iri

'http://xmlns.com/foaf/0.1/Person'

... and "orcid" is a predicate defined by the metadata4ing ontology:

In [14]:
orcid_iri

'http://w3id.org/nfdi4ing/metadata4ing#orcid'

### 3.1 Find data based on IRIs

In [15]:
import rdflib.graph as g

graph = g.Graph()
graph.parse('hdf_meta.jsonld', format='json-ld')

<Graph identifier=Naf54dcb7f7eb432db704441c188d7b9d (<class 'rdflib.graph.Graph'>)>

Note, that we need to provide the PREFIXES, if the json-ld data/file does not include the context.

In [16]:
res = graph.query("""
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX m4i: <http://w3id.org/nfdi4ing/metadata4ing#>

SELECT ?id ?orcid
WHERE {
    ?id a foaf:Person .
    ?id m4i:orcid ?orcid .
    }
""")

In [17]:
for r in res:
    print(r)

(rdflib.term.BNode('Nb3d3d7c4fdf64edf8f6ca02bfadece4b'), rdflib.term.Literal('https://orcid.org/0000-0001-8729-0482', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#anyURI')))


## 4. Examples:

### 4.1 Read metadata from JSON and write to HDF5

Suppose we want to store information about the used software to the HDF5 file. It exists as a JSON-LD file based on the codemeta ontolog. For this example, we use the *h5rdmtoolbox* codemeta.json file from the github repository:

In [18]:
from h5rdmtoolbox.utils import download_file
from pprint import pprint

Download the file:

In [19]:
codemeta_url = 'https://raw.githubusercontent.com/matthiasprobst/h5RDMtoolbox/main/codemeta.json'
dowloaded_filename = download_file(codemeta_url, None)

C:\Users\Matthias\Documents\programming\GitHub\h5RDMtoolbox\h5rdmtoolbox\utils.py:57: UserWarning: No hash given! This is recommended when downloading files from the web.
  warnings.warn('No hash given! This is recommended when downloading files from the web.', UserWarning)


Read the data with `ontolutils.dquery`:

In [20]:
from ontolutils import dquery

In [21]:
data = dquery(subject='schema:SoftwareSourceCode',
              source=dowloaded_filename,
              context={"schema": "http://schema.org/"})
pprint(data[0])

{'@context': {'applicationCategory': 'http://schema.org/applicationCategory',
              'author': 'http://schema.org/author',
              'codeRepository': 'http://schema.org/codeRepository',
              'description': 'http://schema.org/description',
              'license': 'http://schema.org/license',
              'name': 'http://schema.org/name',
              'operatingSystem': 'http://schema.org/operatingSystem',
              'programmingLanguage': 'http://schema.org/programmingLanguage',
              'version': 'http://schema.org/version'},
 '@id': '_:N9a94fa6d2573414388fc455916b3087f',
 '@type': 'http://schema.org/SoftwareSourceCode',
 'applicationCategory': 'file:///C:/Users/Matthias/AppData/Local/h5rdmtoolbox/h5rdmtoolbox/tmp/tmp_0/Engineering',
 'author': [{'@id': 'https://orcid.org/0000-0002-4116-0065',
             '@type': 'http://schema.org/Person',
             'affiliation': {'@type': 'http://schema.org/Organization',
                             'name': 'Ka

The data are written into the HDF5 file by using `jsonld.to_hdf()`:

In [22]:
with h5tbx.File('test.hdf', 'w') as h5:
    jsonld.to_hdf(data=data[0],
                 grp=h5.create_group('software_code'))
    h5.dump(False)

In [23]:
import ontolutils

In [24]:
with h5tbx.File(mode='w') as h5:
    _ = h5.create_dataset('test_dataset', data=np.array([[1, 2], [3, 4], [5.4, 1.9]]))
    _ = h5.create_dataset('test_dataset 2', data=4.5)
    h5.create_dataset('grp/subgrp/vel', data=4)
    h5.attrs['name', ontolutils.SCHEMA.name] = 'test attr'
    # _ = h5.create_dataset('test_dataset', data=5.4)
    jd = jsonld.dumpd(h5, structural=True)
    jds = jsonld.dumps(h5, structural=True, indent=2)
from pprint import pprint
pprint(jd, indent=1)

{'@context': {'hdf5': 'http://purl.allotrope.org/ontologies/hdf5/1.8#'},
 '@graph': [{'@id': '_:N5a2bd304889a490a9f3406620007328a',
             '@type': 'hdf5:File',
             'hdf5:rootGroup': {'@id': '_:N403dcaf1c55742cd9f07a9b8864a7021',
                                '@type': 'hdf5:Group',
                                'hdf5:attribute': {'@id': '_:Nf21f55e6949146e5a15a392e70428690',
                                                   '@type': 'hdf5:Attribute',
                                                   'hdf5:name': 'name',
                                                   'hdf5:value': 'test attr'},
                                'hdf5:member': [{'@id': '_:Nf72bc06efa8a4ec7b0e6939528b0b908',
                                                 '@type': 'hdf5:Group',
                                                 'hdf5:member': {'@id': '_:N5a91f0d585eb491bbcef04400fac19b4',
                                                                 '@type': 'hdf5:Group',
        

## Describing attribute meanings without RDF

Sometimes, there is no IRI (yet) defined but the need to give an additional comment on the attribute. This can be done by as follows:

In [32]:
with h5tbx.File() as h5:
    grp = h5.create_group('contact')

    # Set an attribute as usual
    grp.attrs['type'] = 'Contact'

    # Update the attribute definition afterwards:
    grp.attrsdef['type'] = 'The role of the Person'

    # Alternatively, it can be assigned simultaneously via h5tbx.Attribute:
    grp.attrs['name'] = h5tbx.Attribute(value='Matthias',
                                        definition='The first name of the contact')
    h5.dump(False)